# EERIE Data at DKRZ

This notebook guides EERIE data users and explains how to find and load data available at DKRZ.

All data relevant for the project is referenced in the main DKRZ-EERIE Catalog:

In [1]:
import intake
eerie_cat=intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
eerie_cat

eerie:
  args:
    path: https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


We use a catalog reference syntax, i.e. a path name template:

**hpc.hardware.product.model_id.experiment_id.realm.grid_type**

In Nov 2023, we have the following Controlled Vocabulary:

- hpc
    - dkrz
    - jasmin
- hardware
    - disk
    - cloud
    - archive
- product
    - observations
    - model-output
- model_id
    - icon-esm-er
    - ifs-fesom2-sr
- experiment_id
    - eerie-control-1950
    - eerie-control-spinup
- realm
    - atmos
    - ocean
    - land
- grid_type
    - gr025
    - gr100
    - native
    
Not all combinations are available. But as an example, you can find

In [2]:
eerie_cat["dkrz.disk.model-output.icon-esm-er.eerie-control-1950.atmos.gr025"]

gr025:
  args:
    path: https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/icon-esm-er/eerie-control-1950/atmos/gr025/main.yaml
  description: This catalog contains atmospheric EERIE ICON-ESM-ER eerie-control-1950
    output on 0.25deg grid available at DKRZ disk.
  driver: intake.catalog.local.YAMLFileCatalog
  metadata:
    catalog_dir: https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/icon-esm-er/eerie-control-1950/atmos


Use `list` to explore the content of a catalog

In [3]:
list(eerie_cat)

['jasmin', 'dkrz']

You can print a short description of each collection with its `describe` function:

In [4]:
for col in eerie_cat:
    print(f"Description of {col}:")
    print(eerie_cat[col].describe()["description"])

Description of jasmin:
This catalog contains datasets for EERIE stored on JASMIN
Description of dkrz:
This catalog contains datasets for EERIE stored on DKRZ


Datasets saved on DKRZ are in the DKRZ catalog:

In [5]:
eerie_dkrz=eerie_cat["dkrz"]
for col in eerie_dkrz:
    print(f"Description of {col}:")
    print(eerie_dkrz[col].describe()["description"])

Description of disk:
Use this catalog if you are working on Levante. This catalog contains datasets for all raw data in /work/bm1344 and accesses the data via kerchunks in /work/bm1344/DKRZ/kerchunks.
Description of archive:
Only use this catalog if your desired data is not available on diks and needs to be retrieved from DKRZ's tape archive. This catalog contains datasets for archived data in /arch/bm1344
Description of cloud:
Use this catalog if you are NOT working on Levante. This catalog contains the same datasets as *dkrz_eerie_kerchunk* but data access is via the xpublish server *eerie.cloud.dkrz.de*
Description of main:
DKRZ master catalog for all /pool/data catalogs available
Description of dkrz_ngc3:
NextGEMs Cycle 3 data


The DKRZ catalogs distinguish between the storage location on DKRZ. 

```{note}

If you are accessing the data from remote, you can use the *cloud* catalog. For accessing other data, you have to be logged in on DKRZ´s HPC.

``` 

In [6]:
eerie_dkrz_disk=eerie_dkrz["disk"]
for col in eerie_dkrz_disk:
    print(f"Description of {col}:")
    print(eerie_dkrz_disk[col].describe()["description"])

Description of model-output:
EERIE Earth System Model output available on DKRZ's Levante File System. This catalog contains datasets for all raw data in /work/bm1344 and accesses the data via kerchunks in /work/bm1344/DKRZ/kerchunks
Description of observations:
This catalog contains observational data that is used for EERIE evaluation.


We continue to work with the eerie-dkrz-disk-esm catalog to show how to

- browse
- open and load
- subset

data.

In [7]:
cat=eerie_dkrz_disk["model-output"]["icon-esm-er"]
list(cat)

['eerie-control-1950', 'eerie-spinup-1950']

## Hackathon 2023

In Nov 2023, we have 2 simulations from 2 models available at DKRZ. These simulations are ongoing and are prolonged during 2023.

- icon-esm-er:
    - eerie-spinup-1950 is a combination of erc1007+erc1008
    - eerie-control-1950 is equal to the simulation_id erc1011
        - is the control run with forcing from 1950
- ifs-fesom2-sr:
    - eerie-control-1950
    - IFS4-4-FESOM-5-cycle 3
        - is FESOM output of NextGEMs cycle 3

## Browse

Intake offers users a free text search field. We can search for example for the control run. Intake returns another catalog.

In [8]:
searchdict=dict(
    model="ICON",
    realm="atmos",
    exp="eerie-control-1950",
    var="temperature",
    frequency="monthly"
)
subcat=cat
for v in searchdict.values():
    subcat=subcat.search(v)
list(subcat)

#note that `search` has a keyword argument *depth* (default: 2) which indicates how many subcatalogs should be searched. 
#if you use a high level catalog, adapt that argument to your needs

https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/icon-esm-er/eerie-spinup-1950/atm/main.yaml
https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/icon-esm-er/eerie-spinup-1950/oce/main.yaml
https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/icon-esm-er/eerie-spinup-1950/lnd/main.yaml


['eerie-control-1950.atmos.gr025.2d_monthly_mean',
 'eerie-control-1950.atmos.gr025.model-level_monthly_mean',
 'eerie-control-1950.atmos.native.2d_monthly_mean',
 'eerie-control-1950.atmos.native.model-level_monthly_mean',
 'eerie-control-1950.atmos.native.plev19_monthly_mean']

## Open and load

You can open a dataset from the catalog similar to a dictionary entry. This gives you a lot of metadata information:

In [9]:
cat["eerie-control-1950.atmos.gr025.2d_monthly_mean"]

2d_monthly_mean:
  args:
    chunks: auto
    consolidated: false
    urlpath: reference:://work/bm1344/DKRZ/kerchunks/erc1011/atm_2d_1mth_mean_remap025.json
  description: ''
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    CDI: Climate Data Interface version 2.2.4 (https://mpimet.mpg.de/cdi)
    CDO: Climate Data Operators version 2.2.2 (https://mpimet.mpg.de/cdo)
    Conventions: CF-1.6
    DOKU_License: CC BY 4.0
    DOKU_Name: EERIE ICON Control run
    DOKU_authors: "Putrasahan, D.; Kr\xF6ger, J.; Wachsmann, F."
    DOKU_responsible_person: Fabian Wachsmann
    DOKU_summary: EERIE ICON Control run
    catalog_dir: https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/icon-esm-er/eerie-control-1950/atmos/gr025
    cdo_openmp_thread_number: 16
    comment: Sapphire Dyamond (k203123) on l40335 (Linux 4.18.0-372.32.1.el8_6.x86_64
      x86_64)
    experiment: erc1011
    experiment_id: eerie-1950control
    format: netcdf
    frequency:

You can load the data with the `to_dask()` function.

In [10]:
ds=cat["eerie-control-1950.atmos.gr025.2d_monthly_mean"].to_dask()
ds

<xarray.Dataset>
Dimensions:   (time: 153, lat: 721, lon: 1440, height: 1, height_2: 1)
Coordinates:
  * height    (height) float64 2.0
  * height_2  (height_2) float64 10.0
  * lat       (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * lon       (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * time      (time) datetime64[ns] 2002-02-28T23:59:00 ... 2014-11-30T23:59:00
Data variables: (12/32)
    clivi     (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    cllvi     (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    clt       (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    dew2      (time, height, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    evspsbl   (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    hfls      (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    ...        ...
    tasmin    (time, height, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    tauu      (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    tauv      (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    ts        (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    uas       (time, height_2, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    vas       (time, height_2, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
Attributes: (12/27)
    CDI:                       Climate Data Interface version 2.2.4 (https://...
    CDO:                       Climate Data Operators version 2.2.2 (https://...
    Conventions:               CF-1.6
    DOKU_License:              CC BY 4.0
    DOKU_Name:                 EERIE ICON Control run
    DOKU_authors:              Putrasahan, D.; Kröger, J.; Wachsmann, F.
    ...                        ...
    references:                see MPIM/DWD publications
    simulation_id:             erc1011
    source:                    git@gitlab.dkrz.de:icon/icon-mpim.git@45022778...
    source_id:                 ICON-ESM
    time_reduction:            mean
    title:                     ICON simulation

If data is ICON data on native grid, you should start with setting `lat` and `lon` to coordinates:

In [11]:
ds=ds.set_coords(["lat","lon"])
ds

<xarray.Dataset>
Dimensions:   (time: 153, lat: 721, lon: 1440, height: 1, height_2: 1)
Coordinates:
  * height    (height) float64 2.0
  * height_2  (height_2) float64 10.0
  * lat       (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * lon       (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * time      (time) datetime64[ns] 2002-02-28T23:59:00 ... 2014-11-30T23:59:00
Data variables: (12/32)
    clivi     (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    cllvi     (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    clt       (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    dew2      (time, height, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    evspsbl   (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    hfls      (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    ...        ...
    tasmin    (time, height, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    tauu      (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    tauv      (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    ts        (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    uas       (time, height_2, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    vas       (time, height_2, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
Attributes: (12/27)
    CDI:                       Climate Data Interface version 2.2.4 (https://...
    CDO:                       Climate Data Operators version 2.2.2 (https://...
    Conventions:               CF-1.6
    DOKU_License:              CC BY 4.0
    DOKU_Name:                 EERIE ICON Control run
    DOKU_authors:              Putrasahan, D.; Kröger, J.; Wachsmann, F.
    ...                        ...
    references:                see MPIM/DWD publications
    simulation_id:             erc1011
    source:                    git@gitlab.dkrz.de:icon/icon-mpim.git@45022778...
    source_id:                 ICON-ESM
    time_reduction:            mean
    title:                     ICON simulation

## Subset

Use
- isel for index selection
- sel for value selection. Note that you can use `method=nearest` to do a nearest neighbour interpolation
- groupby for statistics

In [13]:
# get the latest time step
tas_last_timestep=ds["tas"].isel(time=-1)
tas_last_timestep

<xarray.DataArray 'tas' (height: 1, lat: 721, lon: 1440)>
dask.array<getitem, shape=(1, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * height   (height) float64 2.0
  * lat      (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
    time     datetime64[ns] 2014-11-30T23:59:00
Attributes:
    long_name:      temperature in 2m
    param:          0.0.0
    standard_name:  tas
    units:          K

In [14]:
#select a coordinate by values and with nearest neighbor look up:
tas_last_timestep_northsea=ds["tas"].sel(**dict(lat="54",lon="8.2"),method="nearest")
tas_last_timestep_northsea.hvplot.line()

:DynamicMap   [height]
   :Curve   [time]   (tas)

In [15]:
tas_last_timestep_northsea_yearmean=tas_last_timestep_northsea.groupby("time.year").mean()

In [16]:
tas_last_timestep_northsea_yearmean.hvplot.line()

:DynamicMap   [height]
   :Curve   [year]   (tas)